In [1]:
from google.colab import drive
# 挂载谷歌云盘
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd

# 定义文件路径
file_path = '/content/drive/MyDrive/reassignment/reassignment_eeFirm_invorScore_eeAssigneeScore_oreeScore_MA.csv'

# 使用read_csv加载指定的列
df = pd.read_csv(file_path)
print(len(df))

<ipython-input-8-34e7d1aa1c92>:7: DtypeWarning: Columns (5,6,18,25,26,32,34,35,36,66,67,68,69,70,71,72,73) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


1650677


In [9]:
print(df.columns)

Index(['rf_id', 'file_id', 'cname', 'caddress_1', 'caddress_2', 'caddress_3',
       'caddress_4', 'reel_no', 'frame_no', 'convey_text', 'record_dt',
       'last_update_dt', 'page_count', 'purge_in', 'convey_ty',
       'employer_assign', 'or_name', 'exec_dt', 'ack_dt', 'title', 'lang',
       'appno_doc_num', 'appno_date', 'appno_country', 'pgpub_doc_num',
       'pgpub_date', 'pgpub_country', 'grant_doc_num', 'grant_date',
       'grant_country', 'ee_name', 'ee_address_1', 'ee_address_2', 'ee_city',
       'ee_state', 'ee_postcode', 'ee_country', 'index', 'ee_country_full',
       'standard_name_ee', 'asstype_firm_ee', 'asstype_indiv_ee',
       'asstype_government_ee', 'asstype_hosp_ee', 'asstype_university_ee',
       'asstype_institute_ee', 'uspto_StandardName', 'uspto_StemName',
       'index_eeFirm', 'inventor_name', 'standard_inventor_name',
       'standard_or_name', 'max_similarity_invor', 'max_inventor',
       'StandardAssignee_patentsview', 'max_similarity_eeAssignee',
  

先筛选出存在首次转让情况的rf_id，这不应该出现在patent_transaction的rf_id中

In [10]:
import pandas as pd
import pickle

# 假设 df 是已经加载的 DataFrame 并包含相关列

# 筛选不同条件下的 rf_id
conditions = [
    (df['max_similarity_invor'] >= 0.8),
    (df['max_similarity_eeAssignee'] >= 0.8),
    (df['similarity_oree'] >= 0.8),
    (df['is_MA'] == 1)
]

# 提取满足条件的 rf_id
rf_ids = pd.concat([df.loc[cond, 'rf_id'] for cond in conditions]).unique()

# 加载先前保存的 firstassign_rfid1 列表
with open('/content/drive/MyDrive/addition_list/firstassign_rfid1.pkl', 'rb') as file:
    firstassign_rfid1 = pickle.load(file)

# 合并当前找到的 rf_ids 和 firstassign_rfid1 列表，并去除重复项
firstassign_rfid = list(set(firstassign_rfid1).union(set(rf_ids)))

# 将最终的列表保存到新的文件中
with open('/content/drive/MyDrive/addition_list/firstassign_rfid_total.pkl', 'wb') as file:
    pickle.dump(firstassign_rfid, file)

print("最终的 rf_id 列表已保存，并去除了重复项。")


最终的 rf_id 列表已保存，并去除了重复项。


In [11]:
# 筛选满足所有条件的样本
filtered_df = df[
    (df['asstype_firm_ee'] == 1) &
    (df['max_similarity_invor'] < 0.8) &
    (df['max_similarity_eeAssignee'] < 0.8) &
    (df['similarity_oree'] < 0.8) &
    (df['is_MA'] == 0)
]
print(len(filtered_df))
print(len(df))

1315730
1650677


In [12]:
import pandas as pd
import pickle

# 加载之前保存的 firstassign_rfid 列表
with open('/content/drive/MyDrive/addition_list/firstassign_rfid_total.pkl', 'rb') as file:
    firstassign_rfid = pickle.load(file)

# 假设 df2 是已经加载的 DataFrame 并包含列 rf_id
# 剔除 df2 中的 rf_id 在 firstassign_rfid 列表中的样本
final_df = filtered_df[~filtered_df['rf_id'].isin(firstassign_rfid)]

print(len(final_df))
print(len(filtered_df))

1195343
1315730


In [13]:
# 将筛选后的 DataFrame 保存为 CSV 文件，不包含索引
final_df.to_csv('/content/drive/MyDrive/reassignment/reassignment_eeFirm_invorScore_eeAssigneeScore_oreeScore_MA_Save.csv', index=False)

时间限定为2014-2021

In [14]:
import pandas as pd

# 定义文件路径
file_path = '/content/drive/MyDrive/reassignment/reassignment_eeFirm_invorScore_eeAssigneeScore_oreeScore_MA_Save.csv'

# 使用read_csv加载指定的列
final_df = pd.read_csv(file_path)
print(len(final_df))

<ipython-input-14-31f0eb3cd6a7>:7: DtypeWarning: Columns (6,18,25,26,32,35,36,66,67,68,69,70,71,72,73) have mixed types. Specify dtype option on import or set low_memory=False.
  final_df = pd.read_csv(file_path)


1195343


In [15]:
import pandas as pd

# 假设 final_df 已经加载，并且 exec_dt 列可能不是日期时间格式，我们首先将其转换为日期时间格式
final_df['exec_dt'] = pd.to_datetime(final_df['exec_dt'], errors='coerce')

# 筛选 exec_dt 对应年份在 2014 至 2022 之间的样本
final_df = final_df[final_df['exec_dt'].dt.year.between(2014, 2022)]

# 查看筛选后的 DataFrame
print(len(final_df))

546653


In [16]:
# 将筛选后的 DataFrame 保存为 CSV 文件，不包含索引
final_df.to_csv('/content/drive/MyDrive/Patent_transaction/transaction.csv', index=False)

接下来，可以去merge PatentInformation了